In [1]:
!pip install mlflow boto3 awscli

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 91.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.1/140.1 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 111.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.0/14.0 MB 122.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 570.5/570.5 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 703.

In [3]:
import mlflow
#Setup MLflow tracking server
mlflow.set_tracking_uri("http://18.117.193.162:5000/")

#set experiment
mlflow.set_experiment("Exp 4- Handling Imbalanced Data")

2025/08/21 17:38:14 INFO mlflow.tracking.fluent: Experiment with name 'Exp 4- Handling Imbalanced Data' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://campusxalidvcbucket/871019091088766058', creation_time=1755797894700, experiment_id='871019091088766058', last_update_time=1755797894700, lifecycle_stage='active', name='Exp 4- Handling Imbalanced Data', tags={}>

In [4]:
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTEENN
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [5]:
df = pd.read_csv('reddit_preprocessing.csv').dropna(subset=['clean_comment'])
df.shape

(36662, 2)

In [7]:
#Step1: Function to run the experiment
def run_imbalanced_experiment(imbalance_method):
  ngram_range = (1,3)
  max_features = 1000

  #Step2: Train-test split before vectorization and resampling
  X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

  #Step3: Vectorization using TF-Idf, fit on training data only
  vectorizer = TfidfVectorizer(ngram_range = ngram_range, max_features = max_features)
  X_train_vec = vectorizer.fit_transform(X_train)
  X_test_vec = vectorizer.transform(X_test)

  #Step4: Handle class imbalance based on the selected method
  if imbalance_method == 'class_weights':
    class_weight = 'balanced'
  else:
    class_weight = None
    if imbalance_method == 'oversampling':
      smote = SMOTE(random_state=42)
      X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)
    elif imbalance_method == 'adasyn':
      adasyn = ADASYN(random_state = 42)
      X_train_vec, y_train = adasyn.fit_resample(X_train_vec, y_train)
    elif imbalance_method =='undersampling':
      runs = RandomUnderSampler(random_state=42)
      X_train_vec, y_train = runs.fit_resample(X_train_vec, y_train)
    elif imbalance_method == 'smote_enn':
      smote_enn = SMOTEENN(random_state=42)
      X_train_vec, y_train = smote_enn.fit_resample(X_train_vec, y_train)

  #Step5: Define and train a Random Forest Model
  with mlflow.start_run():
    #set tag
    mlflow.set_tag("mlflow.runName", f"Imbalance_{imbalance_method}_RandomForest_TFIDF_Trigrams")
    mlflow.set_tag("experiment_type", "imbalance_handling")
    mlflow.set_tag("model_type", "RandomForestClassifier")

    #Add a description
    mlflow.set_tag("description", f"RandomForest with TF-IDF Trigrams, imbalance handling method={imbalance_method}")

    #Log vectorizer parameter
    mlflow.log_param("vectorizer_type", "TF-IDF")
    mlflow.log_param("ngram_range", ngram_range)
    mlflow.log_param("vectorizer_max_features", max_features)

    #Log Random Forest Parameters
    n_estimators = 200
    max_depth = 5

    mlflow.log_param("n_estimators", n_estimators)
    mlflow.log_param("max_depth", max_depth)
    mlflow.log_param("imbalance_method", imbalance_method)

    #Initialize and train the model
    model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42, class_weight=class_weight)
    model.fit(X_train_vec, y_train)

    #Step6: Make predictions and log metircs
    y_pred = model.predict(X_test_vec)

    accuracy = accuracy_score(y_test, y_pred)

    #Log metrics

    mlflow.log_metric("accuracy", accuracy)

    # Log classification report
    classification_rep = classification_report(y_test, y_pred, output_dict=True)
    for label, metrics in classification_rep.items():
        if isinstance(metrics, dict):
            for metric, value in metrics.items():
                mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
    conf_matrix = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title(f"Confusion Matrix: TF-IDF Trigrams, Imbalance={imbalance_method}")
    confusion_matrix_filename = f"confusion_matrix_{imbalance_method}.png"
    plt.savefig(confusion_matrix_filename)
    mlflow.log_artifact(confusion_matrix_filename)
    plt.close()

    mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_trigrams_imbalance_{imbalance_method}")

#Step7: Run experiments
imbalance_methods = ['class_weight', 'oversampling', 'adasyn', 'undersampling', 'smote_enn']
for method in imbalance_methods:
  run_imbalanced_experiment(method)









2025/08/21 18:00:12 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/21 18:00:21 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_class_weight_RandomForest_TFIDF_Trigrams at: http://18.117.193.162:5000/#/experiments/871019091088766058/runs/7ce68fae37cb4e76b43d1f8ef86468a8
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/871019091088766058


2025/08/21 18:00:44 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/21 18:00:51 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_oversampling_RandomForest_TFIDF_Trigrams at: http://18.117.193.162:5000/#/experiments/871019091088766058/runs/37e8a7a5914943428a4e96cafa213b74
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/871019091088766058


2025/08/21 18:01:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/21 18:01:30 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_adasyn_RandomForest_TFIDF_Trigrams at: http://18.117.193.162:5000/#/experiments/871019091088766058/runs/5b57fbdd0ace4114bcab6d125110f6ed
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/871019091088766058


2025/08/21 18:01:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/08/21 18:01:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Imbalance_undersampling_RandomForest_TFIDF_Trigrams at: http://18.117.193.162:5000/#/experiments/871019091088766058/runs/a05c54afcd184c338a40df5f9f0af002
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/871019091088766058


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
2025/08/21 18:02:46 WARNING mlflow.models.model: 

🏃 View run Imbalance_smote_enn_RandomForest_TFIDF_Trigrams at: http://18.117.193.162:5000/#/experiments/871019091088766058/runs/99f8d3c88bbf405fa81215881cc9f338
🧪 View experiment at: http://18.117.193.162:5000/#/experiments/871019091088766058
